In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install GPUtil
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
#!pip install wandb
import json
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy

import numpy as np
import pandas as pd


import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
import torch

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)

torch.manual_seed(RANDOM_SEED)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 15.5 MB/s 
     |████████████████████████████████| 6.6 MB 65.4 MB/s 
     |████████████████████████████████| 596 kB 67.8 MB/s 
     |████████████████████████████████| 101 kB 13.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 14.4 MB/s 
     |████████████████████████████████| 212 kB 71.6 MB/s 
     |████████████████████████████████| 115 kB 56.0 MB/s 
     |████████████████████████████████| 141 kB 77.5 MB/s 
     |████████████████████████████████| 127 kB 76.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully

In [3]:

def ConvertLabel2ModelLabel(label):
  if label == 'Positive':
    o = 2
  elif label == 'Neutral':
    o = 1
  elif label == 'Negative':
    o = 0
  elif label == 'spam':
    o = 1
  else: 
    print(" error at ConvertLabel2ModelLabel label is " , label)
    o = np.nan
  return o


from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

In [4]:
train_path = '/content/drive/MyDrive/fyp/fyp2/final models and datasets/dataset/dataset_new_with_self_label_final_5.0:3.0:3.0/train_dataset.csv'
valid_path = '/content/drive/MyDrive/fyp/fyp2/final models and datasets/dataset/dataset_new_with_self_label_final_5.0:3.0:3.0/valid_dataset.csv'
model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
#out = '/content/drive/MyDrive/fyp/fyp2/final models and datasets/sva_eval'
out = "/content/temp"

import shutil

if not os.path.exists(out):
  
  # Create a new directory because it does not exist 
  os.makedirs(out)
else:
  shutil.rmtree(out)
#shutil.rmtree(out)

name_of_text_column = "text"
hyperparameter_tuning = True
reduce_ = False
num_hyper_trails = 300
hyper_size = 500
epoch = 10

df_train = pd.read_csv(train_path)
df_valid = pd.read_csv(valid_path)

display(df_train)
display(df_valid)

,Unnamed: 0,text,label,source,quality
0,1027,_Matt African variant can cost bitcoin to cras...,Positive,new_turk_low,low
1,1437,"100k Bitcoin is inevitable, after 100k it's un...",Negative,new_turk_low,low
2,699,Full thanks to as they have analysed this Bit...,Neutral,new_turk_high,high
3,1861,If Bitcoin closes with the weekly candle abov...,Positive,new_turk_low,low
4,497,12 $BTC sold into the bid @ 34111 👈 Bitcoin...,Neutral,new_turk_high,high
...,...,...,...,...,...
3147,610,🌹List your token on FegEx. Just DM . ListOnFEG...,Neutral,new_turk_low,low
3148,842,MATIC__Polygon CRYPTO MARKET CRYPTO BUY_|...,Neutral,new_turk_high,high
3149,303,"__crypto _tw 🔥 SanjiInu 🔥 💥 KYC, CHARITY, NF...",Neutral,self_manual,high
3150,1544,Full thanks to as they have analysed this Bit...,Positive,new_turk_low,low


,Unnamed: 0,text,label,source,quality
0,280,_vlz bluesparrow will take over the world T...,Neutral,incomplete_valid,high
1,349,Listing on coinbase couple of months ago .. /...,Neutral,incomplete_valid,high
2,276,"Putting utility and longevity aside, I need to...",Positive,manual,high
3,239,I am very thankful to for knowing me this dan...,Negative,new_turk_high,high
4,395,New intraday forecasts for EURUSD GBPUSD(Free)...,Neutral,incomplete_valid,high
...,...,...,...,...,...
1010,461,Huuuuge News!!! New Utility Income Breakdown i...,Neutral,new_turk_high,high
1011,202,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high
1012,74,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high
1013,264,This zesty faucet from @_bitcoiner is making m...,Neutral,incomplete_valid,high


In [5]:
df_train['label'] = df_train['label'].apply(ConvertLabel2ModelLabel)
df_valid['label'] = df_valid['label'].apply(ConvertLabel2ModelLabel)


display(df_train)
display(df_valid)


,Unnamed: 0,text,label,source,quality
0,1027,_Matt African variant can cost bitcoin to cras...,2,new_turk_low,low
1,1437,"100k Bitcoin is inevitable, after 100k it's un...",0,new_turk_low,low
2,699,Full thanks to as they have analysed this Bit...,1,new_turk_high,high
3,1861,If Bitcoin closes with the weekly candle abov...,2,new_turk_low,low
4,497,12 $BTC sold into the bid @ 34111 👈 Bitcoin...,1,new_turk_high,high
...,...,...,...,...,...
3147,610,🌹List your token on FegEx. Just DM . ListOnFEG...,1,new_turk_low,low
3148,842,MATIC__Polygon CRYPTO MARKET CRYPTO BUY_|...,1,new_turk_high,high
3149,303,"__crypto _tw 🔥 SanjiInu 🔥 💥 KYC, CHARITY, NF...",1,self_manual,high
3150,1544,Full thanks to as they have analysed this Bit...,2,new_turk_low,low


,Unnamed: 0,text,label,source,quality
0,280,_vlz bluesparrow will take over the world T...,1,incomplete_valid,high
1,349,Listing on coinbase couple of months ago .. /...,1,incomplete_valid,high
2,276,"Putting utility and longevity aside, I need to...",2,manual,high
3,239,I am very thankful to for knowing me this dan...,0,new_turk_high,high
4,395,New intraday forecasts for EURUSD GBPUSD(Free)...,1,incomplete_valid,high
...,...,...,...,...,...
1010,461,Huuuuge News!!! New Utility Income Breakdown i...,1,new_turk_high,high
1011,202,Full thanks to as they have analysed this Bit...,2,incomplete_valid,high
1012,74,Full thanks to as they have analysed this Bit...,2,incomplete_valid,high
1013,264,This zesty faucet from @_bitcoiner is making m...,1,incomplete_valid,high


In [6]:
if reduce_:
  df_train = df_train[0:200]

df_train_hyper = df_train[0:hyper_size] 

dataset = ds.dataset(pa.Table.from_pandas(df_valid).to_batches())
### convert to Huggingface dataset
validation_dataset_torch = Dataset(pa.Table.from_pandas(df_valid))

dataset = ds.dataset(pa.Table.from_pandas(df_train).to_batches())
### convert to Huggingface dataset
training_dataset_torch = Dataset(pa.Table.from_pandas(df_train))

dataset = ds.dataset(pa.Table.from_pandas(df_train_hyper).to_batches())
### convert to Huggingface dataset
training_dataset_torch_hyper = Dataset(pa.Table.from_pandas(df_train_hyper))


print(training_dataset_torch)
print(validation_dataset_torch)
print(training_dataset_torch_hyper)

Dataset({
    features: ['Unnamed: 0', 'text', 'label', 'source', 'quality'],
    num_rows: 3152
})
Dataset({
    features: ['Unnamed: 0', 'text', 'label', 'source', 'quality'],
    num_rows: 1015
})
Dataset({
    features: ['Unnamed: 0', 'text', 'label', 'source', 'quality'],
    num_rows: 500
})


In [7]:


def balanced_df(df_):
  #print(df.shape[0])
  df = df_.copy()
  df = df[['text','label']]

  df_pos = df[df['label'] == 2].sample(frac=1).reset_index(drop=True)
  df_neg = df[df['label'] == 0].sample(frac=1).reset_index(drop=True)
  df_neu = df[df['label'] == 1 ].sample(frac=1).reset_index(drop=True)
  

  pos = df_pos.shape[0]
  a = pos
  #print(a)
  neg = df_neg.shape[0]
  b = neg
  #print(b)
  neu = df_neu.shape[0]
  c = neu
  #print(c)
  #print(a+b+c)
  if a < b and a < c :
    smallest = a
  if b < a and b < c :
    smallest = b
  if c < a and c < b :
    smallest = c

  smallest = int(smallest/2)

  li_test = [df_pos[0:smallest],df_neg[0:smallest],df_neu[0:smallest]]
  li_valid = [df_pos[smallest:smallest*2],df_neg[smallest:smallest*2],df_neu[smallest:smallest*2]]
  #print("check")
  #display(df_pos[0:smallest])
  unused_li = [df_pos[smallest*2:],df_neg[smallest*2:],df_neu[smallest*2:]]
  

  return  pd.concat(li_valid,axis=0).sample(frac=1).reset_index(drop=True),pd.concat(li_test,axis=0).sample(frac=1).reset_index(drop=True), pd.concat(unused_li,axis=0).sample(frac=1).reset_index(drop=True)






In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name,model_max_length=512)



In [9]:
def tokenize_function(data):
    return tokenizer(data['text'], padding="max_length", truncation=True,)

print(type(training_dataset_torch))
train_dataset = training_dataset_torch.map(tokenize_function, batched=True)
eval_dataset = validation_dataset_torch.map(tokenize_function, batched=True)
train_dataset_hyper = training_dataset_torch_hyper.map(tokenize_function, batched=True)
print(train_dataset)
print(eval_dataset)
print(train_dataset_hyper)



<class 'datasets.arrow_dataset.Dataset'>


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['Unnamed: 0', 'text', 'label', 'source', 'quality', 'input_ids', 'attention_mask'],
    num_rows: 3152
})
Dataset({
    features: ['Unnamed: 0', 'text', 'label', 'source', 'quality', 'input_ids', 'attention_mask'],
    num_rows: 1015
})
Dataset({
    features: ['Unnamed: 0', 'text', 'label', 'source', 'quality', 'input_ids', 'attention_mask'],
    num_rows: 500
})


In [10]:
import numpy as np
from datasets import load_metric

metric = load_metric("matthews_correlation")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
#model.to(device)

In [12]:
from transformers import Trainer
from transformers import TrainingArguments
#learning_rate = 5e-05
batch_size = 8
#eval_batch_size = 4
seed = 40
#optimizer = Adam 
#with betas=(0.9,0.999) and epsilon=1e-08
adam_beta1 = 0.9
adam_beta2 =0.999
lr_scheduler_type = "linear"
num_epochs = 15
#args = TrainingArguments("test_trainer",report_to="wandb" ,logging_strategy = "epoch",evaluation_strategy="epoch",learning_rate = learning_rate,num_train_epochs = num_epochs,lr_scheduler_type =lr_scheduler_type, adam_beta1 = adam_beta1,adam_beta2 =adam_beta2  )

args = TrainingArguments(
    #'/content/drive/MyDrive/fyp/fyp2/model/model2-supervised/' + f"{model_name}-finetuned-",
    #'/content/' + f"{model_name}-finetuned-",
    out,
    #report_to="wandb",
    overwrite_output_dir = True,
    logging_dir = out,
    evaluation_strategy = "epoch",
    logging_strategy = "epoch",
    #save_strategy = "NO",
    learning_rate=4.5682e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=0.01,
    #metric_for_best_model='matthews_correlation',
    save_total_limit = 2,
    save_strategy = "no",
    load_best_model_at_end = False,
    seed = 40
    #push_to_hub=True,
)




#trainer = Trainer(
   # model=model,
    #args=args,
   # train_dataset=train_dataset,
  #  eval_dataset=eval_dataset,
 #   compute_metrics=compute_metrics,
#)

if hyperparameter_tuning == False:

  trainer = Trainer(
      model,
      args,
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
      tokenizer=tokenizer,
      compute_metrics=compute_metrics
  )
  # training
  train_result = trainer.train() 
  logs = trainer.state.log_history
  # compute train results
  metrics = train_result.metrics
  max_train_samples = len(train_dataset)
  metrics["train_samples"] = min(max_train_samples, len(train_dataset))

  # save train results
  trainer.log_metrics("train", metrics)
  trainer.save_metrics("train", metrics)

  # compute evaluation results
  metrics = trainer.evaluate()
  max_val_samples = len(eval_dataset)
  metrics["eval_samples"] = min(max_val_samples, len(eval_dataset))

  # save evaluation results
  trainer.log_metrics("eval", metrics)
  trainer.save_metrics("eval", metrics)

  import pickle
  save_logs_dir = out
  name_logs = "logs"

  my_file = save_logs_dir + "/" + name_logs + ".p"
  print("savint to " , my_file)

  pickle.dump( logs, open( my_file, "wb" )    ) 

  ld = pickle.load( open( my_file, "rb" ) )


  print(logs)
  print(logs[2])
  print(type(logs))

  print(ld)
  print(type(ld))
  print(logs == ld)
  
  #trainer.evaluate()



In [13]:
#!pip install ray[tune]
!pip install optuna
import optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 14.8 MB/s 
     |████████████████████████████████| 209 kB 63.0 MB/s 
     |████████████████████████████████| 81 kB 10.9 MB/s 
     |████████████████████████████████| 78 kB 7.9 MB/s 
     |████████████████████████████████| 112 kB 71.1 MB/s 
     |████████████████████████████████| 147 kB 75.1 MB/s 
     |████████████████████████████████| 49 kB 7.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=36cd86139e1d649015c8d6f641de90c472b2e9ed17b4e62e76bc9fd4c62e1cb5
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [14]:
!pip install pickle5
import pickle5 as pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 15.3 MB/s 


In [15]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

In [16]:
!nvidia-smi 

Sun Aug  7 00:20:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    24W / 300W |      2MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
if hyperparameter_tuning:
  
  args.load_best_model_at_end = False
  args.save_stratgy = "no"
  #args.out_dir = "/content/hyper_fold"
  #model_args =args
  #model_args.reprocess_input_data = True
  #model_args.overwrite_output_dir = True
  #model_args.evaluate_during_training = True
  #model_args.manual_seed = 4
  #model_args.use_multiprocessing = True


  trainer = Trainer(
      model_init=model_init,
      args=args,
      train_dataset=train_dataset_hyper,
      eval_dataset=eval_dataset,
      tokenizer=tokenizer,
      compute_metrics=compute_metrics
  )

  best_run = trainer.hyperparameter_search(n_trials=num_hyper_trails, direction="maximize")
  best_run
#trainer.train()

#trainer.evaluate()

loading configuration file https://huggingface.co/cardiffnlp/twitter-xlm-roberta-base-sentiment/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/9628a03bf91a381b0f93e02e13ed34077a805ede6a568ad868817f87437a55ea.ea50decabb7db740257ca1cdefd63c25ffafb958ec595a0ff0c8dbac3f4b1ae6
Model config XLMRobertaConfig {
  "_name_or_path": "cardiffnlp/twitter-xlm-roberta-base-sentiment",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Negative",
    "1": "Neutral",
    "2": "Positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Negative": 0,
    "Neutral": 1,
    "Positive": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.960500,0.839276,0.336672


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: source, quality, text, Unnamed: 0. If source, quality, text, Unnamed: 0 are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1015
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)


[I 2022-08-07 00:21:29,794] Trial 0 finished with value: 0.3366717497039904 and parameters: {'learning_rate': 1.0671839972477622e-05, 'num_train_epochs': 1, 'seed': 15, 'per_device_train_batch_size': 8}. Best is trial 0 with value: 0.3366717497039904.
Trial:
loading configuration file https://huggingface.co/cardiffnlp/twitter-xlm-roberta-base-sentiment/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/9628a03bf91a381b0f93e02e13ed34077a805ede6a568ad868817f87437a55ea.ea50decabb7db740257

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.992200,0.850254,0.375627
2,0.911400,0.818366,0.376046
3,0.899700,0.823815,0.371501
4,0.875500,0.819205,0.373184
5,0.886000,0.817568,0.373184


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: source, quality, text, Unnamed: 0. If source, quality, text, Unnamed: 0 are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1015
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: source, quality, text, Unnamed: 0. If source, quality, text, Unnamed: 0 are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1015
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: source, quality, text, Unnamed: 0. If source, qualit

...

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.953300,0.817256,0.115645


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: source, quality, text, Unnamed: 0. If source, quality, text, Unnamed: 0 are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1015
  Batch size = 8
[I 2022-08-07 00:41:53,022] Trial 15 pruned. 
Trial:
loading configuration file https://huggingface.co/cardiffnlp/twitter-xlm-roberta-base-sentiment/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/9628a03bf91a381b0f93e02e13ed34077a805ede6a568ad868817f87437a55ea.ea50decabb7db740257ca1cdefd63c25ffafb958ec595a0ff0c8dbac3f4b1ae6
Model config XLMRobertaConfig {
  "_name_or_path": "cardiffnlp/twitter-xlm-roberta-base-sentiment",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_drop

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.969000,0.849714,0.180839


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: source, quality, text, Unnamed: 0. If source, quality, text, Unnamed: 0 are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1015
  Batch size = 8
[I 2022-08-07 00:42:30,947] Trial 16 pruned. 
Trial:
loading configuration file https://huggingface.co/cardiffnlp/twitter-xlm-roberta-base-sentiment/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/9628a03bf91a381b0f93e02e13ed34077a805ede6a568ad868817f87437a55ea.ea50decabb7db740257ca1cdefd63c25ffafb958ec595a0ff0c8dbac3f4b1ae6
Model config XLMRobertaConfig {
  "_name_or_path": "cardiffnlp/twitter-xlm-roberta-base-sentiment",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_drop

Epoch,Training Loss,Validation Loss


In [ ]:
if hyperparameter_tuning:
  best_run
  try:
    hyper_best = []
    for n, v in best_run.hyperparameters.items():
        t = [n,v]
        hyper_best.append(t)
        print("in")
        print(n)
        print(v)
        print("out")
        setattr(trainer.args, n, v)

    import pickle
    my_file = out + "/hyper_best.p"
    print("saving hyper")
    print(hyper_best)
    pickle.dump( hyper_best, open( my_file, "wb" )   ) 

    ld = pickle.load( open( my_file, "rb" ) )
    print("saved hyper")
    print(ld)

    #args.num_train_epochs = 40
    args.load_best_model_at_end = True
    args.save_stratgy = "epoch"
    args.out_dir = out

    #free_gpu_cache()

    trainer = Trainer(
      model,
      args,
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
      tokenizer=tokenizer,
      compute_metrics=compute_metrics
    )


    train_result = trainer.train() 
  except Exception as e:
    print(e)
    print("fail")
    #!kill -9 -1
    pass


In [ ]:
#!kill -9 -1
!nvidia-smi